# Time In Process
This Jupyter Notebook address the questions mentioned in the prompts for the Ashby take home test.  The questions are:
1. What is the company's Time in Process? Is that good or bad?
1. How is the company's Time in Process trending?
1. How does the company's Time in Process vary across myriad segments?
    - For example, Time in Proccess by department, location, application outcome, etc.
1. What is the average or median Time in Process for specific stages or stage groups?

Additional points to consider:
- It is best to focus results and analysis at the stage group level.
- It is common for a candidate's application to enter a given stage more than once.
- While there is no bad data, data can be removed if it appears erroneous, but documentation is required for the removal.
- Customer Success team doesn't have the ability to create their own datasets or visualization content, but can apply filters or updates to content provided for them.

## What is the company's Time in Process?

Time in Process refers to the time a candidate takes to move through the interview pipeline. Time in Process (TiP) can be thought of as time in a specific stage or the total time spent in the process. TiP ends when a candidate is either archived or hired (however, a candidate can be unarchived).

To answer this question, we want to find the average/median TiP. Finding the sum of TiP doesn't really make sense for this question, as it doesn't reveal anything about the process. Understanding the average/median TiP however, can helps to discover potential roadblocks for the job itself, or for the individual stages.


The questions for the Ashby assignment can be answered by the following:
1. Overall TiP
    - Application status must be archived or hired for the overal TiP
        - Exclude applicants that are not completed because they may skew the result
    1. Total TiP for archived and hired applicants
        1. Filter by `stg_applications.status in ('archived', 'hired')`
        1. Join `stg_applications.id = stg_stage_transitions.application_id`
        1. Join `stg_interview_stages_and_groups.stage_id = stg_stage_transitions.new_stage_id`
        1. Filter by `stg_interview_stages_and_groups.stage_group_type in ('archived', 'hired')`
1. TiP per Stage
    - Look at all stage transitions that have a left_stage_at column in stage_transactions
    - May want to look at visuals for all applicants and also for applicants that have completed TiP.
    - Even if an applicant goes back to a prior stage, assume that the stages are unique (count the stages as individual stages).
        - Since going back to a prior stage may be the company's intention and not a mistake, it is not possible to assume it was a mistake.
    1. Join `stg_applications.id = stg_stage_transitions.application_id`
        - Get application data to understand change over time.
    1. Join `stg_stage_transitions.new_stage_id = stg_interview_stages_and_groups.stage_id`
    1. Filter `stg_stage_transitions.left_stage_at IS NOT NULL`
        - Most likely will want to filter applicants that are not finished in the stage

The other questions can be answered by the proper grouping of dimensions.

In [74]:
import duckdb

db = duckdb.connect("../../database/ashby.db", read_only = True)


In [42]:
db.sql("select * from stg_interview_stages_and_groups").show()
db.close()

ConnectionException: Connection Error: Connection has already been closed

In [38]:
# Application
# archived
# 0ba8eb29-b003-4f4c-806c-be150b2c731b 
# hired
# 5ffad0ba-27b7-4156-8751-bb7d394434dc
# negative application
# e60fb77e-da82-443d-9f84-960c3085b82a
db.sql("""
    SELECT 
        *
    FROM stg_applications
    where id = 'e60fb77e-da82-443d-9f84-960c3085b82a'
    limit 10
    """).show(max_width=10000)

┌──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬────────────────────────────┬──────────────────────────┬──────────┐
│                  ID                  │          ARCHIVE_REASON_ID           │           ORGANIZATION_ID            │                JOB_ID                │             CANDIDATE_ID             │      CURRENT_INTERVIEW_STAGE_ID      │              SOURCE_ID               │         created_at         │     last_activity_at     │  status  │
│               varchar                │               varchar                │               varchar                │               varchar                │               varchar                │               varchar                │               varchar                │  timestamp with time zone  │ timestamp wi

In [39]:
# Stage Transitions
# archived
# 0ba8eb29-b003-4f4c-806c-be150b2c731b
# hired
# 5ffad0ba-27b7-4156-8751-bb7d394434dc
# negative application
# e60fb77e-da82-443d-9f84-960c3085b82a
db.sql("""
    SELECT 
        *
    FROM stg_stage_transitions
    where application_id = 'e60fb77e-da82-443d-9f84-960c3085b82a'
    limit 10
    """).show(max_width=10000)

┌──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬────────────────────────────┬────────────────────────────┬───────────────────────────┐
│                  ID                  │            APPLICATION_ID            │        NEW_INTERVIEW_STAGE_ID        │           ORGANIZATION_ID            │     PREVIOUS_INTERVIEW_STAGE_ID      │         created_at         │      entered_stage_at      │       left_stage_at       │
│               varchar                │               varchar                │               varchar                │               varchar                │               varchar                │  timestamp with time zone  │  timestamp with time zone  │ timestamp with time zone  │
├──────────────────────────────────────┼──────────────────────────────────────┼──────────────────────────────────────┼─────────────────────────────────

In [40]:
# Stage Information
# archived
# ('ec228221-57a2-47ec-8d12-e76bb3c11463', 'abe0f89d-8119-41ac-a0e4-44aa2ca0f21f', 'c7441dc6-f3d0-40cc-8404-cce9c5586651')
# hired
# ('636f502a-b51f-4ad0-9f20-b192555ff3ce', 'cbd7f502-b563-4af3-936c-fd89609436ee', 'a7c5c894-b219-4c66-87d5-0571385630d8', 'c5b39dfc-5776-4e7d-aec5-c0457b4167af', '2dfacf06-cd92-4e0f-8029-b47f8890d141')
# negative application
# ('0879eb7a-1191-4656-b318-3ea1cc046e6e', '1c632eda-c66c-4fa7-9ad3-03f77200d01a')
db.sql("""
    SELECT 
        *
    FROM stg_interview_stages_and_groups
    where stage_id in ('3d30407d-bf4d-4cf9-9a70-44d30540c65d', '5889b336-025b-4920-b738-712fb1398883')
    limit 10
    """).show(max_width=10000)

┌──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬────────────────────────────┬───────────────────┬────────────────────┬────────────────────┬─────────────┬────────────────────┬────────────────────┬─────────────┐
│               STAGE_ID               │       INTERVIEW_STAGE_GROUP_ID       │           ORGANIZATION_ID            │            STAGE_GROUP_ID            │         created_at         │ STAGE_GROUP_ORDER │ STAGE_GROUP_TITLE  │  stage_group_type  │ STAGE_ORDER │     STAGE_TYPE     │       TITLE        │ is_archived │
│               varchar                │               varchar                │               varchar                │               varchar                │  timestamp with time zone  │       int64       │      varchar       │      varchar       │    int64    │      varchar       │      varchar       │   boolean   │
├──────────────────────────────────────┼───